In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from scipy import signal
import soundfile as sf
import os

In [3]:
scp = np.loadtxt("dev_as_test.lst", dtype=str)
def read_data(path):
    speech, _ = sf.read(path)
#     print(_)
    data = np.reshape(speech, [1, -1])
    if data.shape[1] % 96000:
        if data.shape[1] * 2 < 96000:
            data = np.tile(data, (1, 96000 // data.shape[1]))
        data = np.hstack([data, data[:, :96000 - data.shape[1] % 96000]])
    data = np.split(data, data.shape[1] / 96000, axis=1)
    data = np.array(data, dtype=np.float32)
    return data
#     print(data.shape)

scp = ["LA_D_1000265.flac", "LA_D_3262859.flac",  "LA_D_3266246.flac", "LA_D_3267906.flac", "LA_D_3260017.flac", "LA_D_3263334.flac", "LA_D_3266332.flac", "LA_D_3268182.flac", "LA_D_3260123.flac", "LA_D_3263579.flac",  "LA_D_3266553.flac", "LA_D_3268836.flac", "LA_D_3260640.flac", "LA_D_3263795.flac", "LA_D_3266637.flac", "LA_D_3269593.flac", "LA_D_3260902.flac", "LA_D_3264000.flac", "LA_D_3266852.flac", "LA_D_3261219.flac", "LA_D_3264696.flac", "LA_D_3267050.flac", "LA_D_3262004.flac", "LA_D_3264952.flac", "LA_D_3267875.flac"]
scp_ = []
for scp__ in scp:
    scp_.append(os.path.join("wav", scp__))
scp = scp_
del scp_
# print(scp)
test_data = []
for i in scp:
    test_data.append(read_data(i))

test_frame_num = len(test_data)
test_data = np.concatenate(test_data, axis=0)

# test_data, scp as test names, test_frame_num
# print(test_data)
print(test_data.shape)

(26, 1, 96000)


In [4]:
genuine_labels = np.zeros((120, 2), dtype=np.float32)
spoof_labels = np.zeros((120, 2), dtype=np.float32)

genuine_labels[:, 0] = 1.0 # [1, 0]
spoof_labels[:, 1] = 1.0 # [0, 1]

labels = np.concatenate((genuine_labels, spoof_labels), axis=0)

print(labels.shape)
print(labels[0])

(240, 2)
[1. 0.]


In [5]:
print(test_data.shape)
inputs = np.reshape(test_data, [26, 1, 96000]).shape

(26, 1, 96000)


In [6]:
a = np.zeros(10)
print(a)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [3]:
import torch

In [8]:
input = torch.randn(64, 256, 31)

In [9]:
# print(input.shape)
conv1 =  torch.nn.Conv1d(256, 86, 9, stride=2, dilation=2, padding=2)
output = conv1(input)
print(output.shape)

# output = output[:,:,:10]
# noise_means = torch.randn(64, 86, 10)
# output_l = output[:,:,output.shape[2]-10:]
# print(type(output))
# output = torch.cat((output, noise_means), 0)
print(output.shape)

pool1 = torch.nn.MaxPool1d(3, stride=1, dilation=1, padding=1)
output = pool1(output)
# print(output.shape)
output = torch.cat((output, output), 0)
print(output.shape)

torch.Size([64, 86, 10])
torch.Size([64, 86, 10])
torch.Size([128, 86, 10])


/home/tomtony/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [10]:
conv2 = torch.nn.Conv1d(86, 29, 4, stride=2, dilation=2, padding=1)
output_ = conv2(output)
print(output_.shape)
pool2 = torch.nn.MaxPool1d(3,stride=1)
output__ = pool2(output_)
print(output__.shape)

torch.Size([128, 29, 3])
torch.Size([128, 29, 1])


In [11]:
import torchaudio
# import os
# # print(os.path.abspath(os.getcwd()))
# # waveform, sample_rate = torchaudio.load("/mnt/c/Users/YH-ANZHE/D1_1002601.wav")
# waveform = test_data[0]
# sample_rate = 16000
# print(waveform.shape, sample_rate)
# stft_result = torch.stft(waveform[:,10:], sample_rate, win_length=10)
# print(stft_result.shape)

In [4]:
waveform = test_data[0]
sample_rate = 16000
import librosa
import math


def extract_features(waveform, sample_rate):
    # print(waveform.numpy())
    print(len(waveform[0]))
    n_fft = 1024
    win_length = int(np.ceil(0.025*sample_rate))
    hop_length = int(np.ceil(0.010*sample_rate))
    window = "hamming"

    n_mels = 256
    fmin = 20
    fmax = 8000
    X = librosa.stft(waveform[0], n_fft, hop_length, win_length, window, center=True)
#     print(np.abs(X).shape)
    frames = np.log(librosa.feature.melspectrogram(y=waveform[0], sr=sample_rate, S=X, n_mels=n_mels, fmin=fmin, fmax=fmax) + 1e-6)
    print(X.shape)




#     noise=np.random.normal(0, 10, waveform.shape[1]).reshape(-1, waveform.shape[1])
#     N = librosa.stft(noise[0], n_fft, hop_length, win_length, window, center=True)
    
# #     print(noise)
#     SNM = 1/(1 + np.exp(-np.abs(X)/np.abs(N)))
#     print(SNM.shape)
#     print(frames.shape)
#     frames = np.log(librosa.feature.melspectrogram(y=waveform[0]+noise[0], sr=sample_rate, S=SNM, n_mels=n_mels, fmin=fmin, fmax=fmax) + 1e-6)
    




    window_size = 31
    window_hop = 31
    start_frame = window_size 
    end_frame = window_hop * math.floor(float(frames.shape[1]) / window_hop/2)
    window = frames[:, end_frame: end_frame+window_size]
    
    noises_features = np.mean(abs(frames[:, :10]), axis=1)
    
    return abs(window), noises_features


window, noises_features = extract_features(waveform, sample_rate)

# window_size = 31
# window_hop = 31
# start_frame = window_size 
# end_frame = window_hop * math.floor(float(frames.shape[1]) / window_hop/2)

    
    
import matplotlib.pyplot as plt


# inputs = []
# window = frames[:, end_frame: end_frame+window_size]
# assert window.shape == (n_mels, window_size)
# inputs.append(abs(window))
plt.imshow(window, cmap='jet', origin='lower', aspect='auto')
plt.colorbar()
plt.show()


# noises = abs(frames[:, :10])
# print(noises.shape)
# noises_features = np.mean(noises, axis=1)
# print(noises_features.shape)
# for frame_idx in range(start_frame, end_frame, window_hop):

#     window = frames[:, frame_idx-window_size:frame_idx]
#     assert window.shape == (n_mels, window_size)
#     print('classify window', frame_idx, window.shape)
# #     print(window)
#     inputs.append(abs(window))
#     plt.imshow(abs(window), cmap='jet', origin='lower', aspect='auto')
#     plt.colorbar()
#     plt.show()

NameError: name 'test_data' is not defined

In [5]:
from torch import nn

cnn_layers = nn.Sequential(
            nn.Conv2d(1, 64, (9,9), stride=(1, 1), padding='same'),
            nn.Sigmoid(),
            nn.MaxPool2d((3,3), stride=(3,3), ceil_mode=False),
            nn.Conv2d(64, 128, (4,4), stride=(1,1), padding='same'),
            nn.Sigmoid(),
            nn.MaxPool2d((3,3), stride=(3, 3), ceil_mode=False),
        )

inputs_ = np.array(inputs)
inputs_ = inputs_.reshape(-1, 1, 256, 31)
# print(inputs_.shape)
inputs_ = torch.from_numpy(inputs_)
# print(inputs.shape)
# inputs = torch.reshape(inputs, (11, 1, 256, -1))
# print(torch.FloatTensor(inputs).shape)
outputs = cnn_layers(torch.tensor(inputs_))

print(outputs.shape)

NameError: name 'inputs' is not defined

In [14]:
fc3 = nn.Sequential(
        nn.Linear(128*28*3+256, 1024),
        nn.Linear(1024, 1024),
        nn.Linear(1024, 256),
    )

inputs__ = torch.cat((outputs[0].flatten(), torch.from_numpy(noises_features).flatten()))
outputs__ = fc3(inputs__)
print(outputs__.shape)
print(outputs__)

NameError: name 'outputs' is not defined

In [6]:
# CNN Network for SNM features 最终版本 219
class CNN_SNM(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 64, (9,9), stride=(1, 1), padding='same'),
            nn.Sigmoid(),
            nn.MaxPool2d((3,3), stride=(3,3), ceil_mode=False),
            nn.Conv2d(64, 128, (4,4), stride=(1,1), padding='same'),
            nn.Sigmoid(),
            nn.MaxPool2d((3,3), stride=(3, 3), ceil_mode=False),
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(128*28*3+256, 1024),
            nn.Linear(1024, 1024),
            nn.Linear(1024, 256),
        )

    def forward(self, x, noises_features):
        result = self.cnn_layers(x)
        result = self.fc3(torch.cat((result[0].flatten(), torch.from_numpy(noises_features).flatten())))
        return result


In [16]:
input_data = test_data[0]
window, noises_features = extract_features(input_data, 16000)
window = window.reshape(-1, 1, 256, 31)
net_SNM = CNN_SNM()
SNMs = net_SNM(torch.tensor(window), noises_features)
# print(SNMs)

# get net_SNM paremeters:
# print(list(net_SNM.parameters()))
for name, parameters in net_SNM.named_parameters():
    print(name, ":", parameters.size())

96000
(513, 601)
cnn_layers.0.weight : torch.Size([64, 1, 9, 9])
cnn_layers.0.bias : torch.Size([64])
cnn_layers.3.weight : torch.Size([128, 64, 4, 4])
cnn_layers.3.bias : torch.Size([128])
fc3.0.weight : torch.Size([1024, 11008])
fc3.0.bias : torch.Size([1024])
fc3.1.weight : torch.Size([1024, 1024])
fc3.1.bias : torch.Size([1024])
fc3.2.weight : torch.Size([256, 1024])
fc3.2.bias : torch.Size([256])


/home/tomtony/anaconda3/lib/python3.8/site-packages/torch/nn/modules/conv.py:439: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  /pytorch/aten/src/ATen/native/Convolution.cpp:660.)
  return F.conv2d(input, weight, bias, self.stride,


In [17]:
mat1 = np.zeros((256, 31))
mat2 = np.zeros((256, 31))

mat_ = mat1/mat2

<ipython-input-17-e46e003b20b3>:4: RuntimeWarning: invalid value encountered in true_divide
  mat_ = mat1/mat2


In [19]:
m = nn.Linear(20, 1000)
input = torch.randn(128, 20)
print(input.shape)
output = m(input)
print(output.shape)

m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
print(input.shape, target.shape)
print(input, m(input), target)
output = loss(m(input), target)
# print(output.shape)
print(output)
output.backward()
print(output)

torch.Size([128, 20])
torch.Size([128, 1000])
torch.Size([3]) torch.Size([3])
tensor([0.8999, 0.7554, 0.7122], requires_grad=True) tensor([0.7109, 0.6803, 0.6709], grad_fn=<SigmoidBackward>) tensor([0., 0., 0.])
tensor(1.1643, grad_fn=<BinaryCrossEntropyBackward>)
tensor(1.1643, grad_fn=<BinaryCrossEntropyBackward>)


/home/tomtony/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:147: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  Variable._execution_engine.run_backward(


In [41]:
with open('../../Downloads/GRCNN-main/train_img_list.pkl', 'rb') as f:
    data = np.load(f, allow_pickle=True)
print("data: ", len(data))
print("data[0]: ", len(data[0]))
print("data[0][0]: ", len(data[0][0]))
print("data[0][0][0]: ", len(data[0][0][0])) # (1000, 1300, 49, 1)

data:  1000
data[0]:  1300
data[0][0]:  49
data[0][0][0]:  1


In [376]:
# # https://gist.github.com/daskol/05439f018465c8fb42ae547b8cc8a77b
# class Maxout(nn.Module):
#     """Class Maxout implements maxout unit introduced in paper by Goodfellow et al, 2013.
    
#     :param in_feature: Size of each input sample.
#     :param out_feature: Size of each output sample.
#     :param n_channels: The number of linear pieces used to make each maxout unit.
#     :param bias: If set to False, the layer will not learn an additive bias.
#     """
    
#     def __init__(self, in_features, out_features, n_channels, bias=True):
#         super().__init__()
        
#         self.in_features = in_features
#         self.out_features = out_features
#         self.n_channels = n_channels

#         print("n_channels * out_features, in_features: ", n_channels * out_features, in_features)

#         self.weight = nn.Parameter(torch.Tensor(n_channels * out_features, in_features))
        
#         if bias:
#             self.bias = nn.Parameter(torch.Tensor(n_channels * out_features))
#         else:
#             self.register_parameter('bias', None)
            
#         self.reset_parameters()
    
#     def forward(self, input):
#         print("input shape: ", input.shape)
#         print("weight shape: ", self.weight.shape)
#         a = nn.functional.linear(input, self.weight, self.bias)
#         b = nn.functional.max_pool1d(a.unsqueeze(-3), kernel_size=self.n_channels)
#         return b.squeeze()
    
#     def reset_parameters(self):
#         irange = 0.005
#         nn.init.uniform_(self.weight, -irange, irange)
#         if self.bias is not None:
#             nn.init.uniform_(self.bias, -irange, irange)
    
#     def extra_repr(self):
#         return (f'in_features={self.in_features}, '
#                 f'out_features={self.out_features}, '
#                 f'n_channels={self.n_channels}, '
#                 f'bias={self.bias is not None}')

from torch.autograd import Function

class Maxout(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input):
        x = input
        max_out=2    #Maxout Parameter
        kernels = x.shape[1]  # to get how many kernels/output
        feature_maps = int(kernels / max_out)
        out_shape = (x.shape[0], feature_maps, max_out, x.shape[2], x.shape[3])
#         print(x.shape)
        x= x.view(out_shape)
#         print(x.shape)
        y, indices = torch.max(x[:, :, :], 2)
        ctx.save_for_backward(input)
        ctx.indices=indices
        ctx.max_out=max_out
        return y

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        input1,indices,max_out= ctx.saved_variables[0],Variable(ctx.indices),ctx.max_out
        input=input1.clone()
        for i in range(max_out):
            a0=indices==i
            input[:,i:input.data.shape[1]:max_out]=a0.float()*grad_output
      

        return input


class ConvGRUCell(nn.Module):
    """
    Generate a convolutional GRU cell
    """

    def __init__(self, input_size, output_size, conv_filter, conv_stride, max_filter, max_stride):
        super().__init__()
#         padding = conv_filter // 2
        self.input_size = input_size
        self.output_size = output_size
        self.reset_gate = nn.Conv2d(input_size, output_size, conv_filter, stride=conv_stride, padding="same")
        self.reset_gate_ = nn.Conv2d(output_size//2, output_size, conv_filter, stride=conv_stride, padding="same")
#         print(hidden_size, hidden_size//2, 3)
        self.reset_maxout = Maxout.apply
        self.update_gate = nn.Conv2d(input_size, output_size, conv_filter, stride=conv_stride, padding="same")
        self.update_gate_ = nn.Conv2d(output_size//2, output_size, conv_filter, stride=conv_stride, padding="same")
        self.update_maxout = Maxout.apply
        self.out_gate = nn.Conv2d(input_size, output_size, conv_filter, stride=conv_stride, padding="same")
        self.out_gate_ = nn.Conv2d(output_size//2, output_size, conv_filter, stride=conv_stride, padding="same")
        self.out_maxout = Maxout.apply
        self.max_pool = nn.MaxPool2d(max_filter, stride=max_stride, ceil_mode=False)
        
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()



        nn.init.orthogonal_(self.reset_gate.weight)
        nn.init.orthogonal_(self.update_gate.weight)
        nn.init.orthogonal_(self.out_gate.weight)
        nn.init.constant(self.reset_gate.bias, 0.)
        nn.init.constant(self.update_gate.bias, 0.)
        nn.init.constant(self.out_gate.bias, 0.)


    def forward(self, input_, prev_state):

        # get batch and spatial sizes
        batch_size = input_.data.size()[0]
        spatial_size = input_.data.size()[2:]
#         print(batch_size, spatial_size)
        
        # generate empty prev_state, if None is provided
        if prev_state is None:
            state_size = [batch_size, self.output_size//2] + list(spatial_size)
            if torch.cuda.is_available():
                prev_state = Variable(torch.zeros(state_size)).cuda()
            else:
                prev_state = Variable(torch.zeros(state_size))

        # data size is [batch, channel, height, width]
        # stacked_inputs = torch.cat([input_, prev_state], dim=1)
        # update = F.sigmoid(self.update_maxout(self.update_gate(stacked_inputs)))
        # reset = F.sigmoid(self.reset_maxout(self.reset_gate(stacked_inputs)))
        # # out_inputs = F.tanh(self.out_gate(torch.cat([input_, prev_state * reset], dim=1)))
#         print(input_.shape, prev_state.shape)
        z_t_n = self.sigmoid(self.update_maxout(self.update_gate(input_)+self.update_gate_(prev_state)))
        r_t_n = self.sigmoid(self.reset_maxout(self.reset_gate(input_)+self.reset_gate_(prev_state)))
#         print("z_t_n.shape, r_t_n.shape, prev_state.shape", z_t_n.shape, r_t_n.shape, prev_state.shape)
#         print("(r_t_n * prev_state).shape", torch.mul(r_t_n, prev_state).shape)
        hidden_ = self.tanh(self.out_maxout(self.out_gate(input_)+self.out_gate_(r_t_n * prev_state)))
#         print(z_t_n.shape, r_t_n.shape, prev_state.shape, hidden_.shape)
        new_state = z_t_n * prev_state + (1 - z_t_n) * hidden_

        return self.max_pool(new_state), hidden_

In [405]:
class GRCNN(nn.Module):
    def __init__(self, classes_num):
        super().__init__()
        self.GRCU_1 = ConvGRUCell(1, 16, (5, 5), (1, 1), (2, 1), (2, 1)) # (input_size, hidden_size, conv_filter, conv_stride, max_filter, max_stride)
        self.GRCU_2 = ConvGRUCell(8, 32, (3, 3), (1, 1), (2, 1), (2, 1))
        self.GRCU_3 = ConvGRUCell(16, 16, (3, 3), (1, 1), (2, 1), (2, 1))
        self.fc = nn.Linear(8*32*32, 480*2)
        self.maxout = Maxout.apply
        self.fc_class = nn.Linear(480, classes_num)
        self.softmax = nn.Softmax(dim=0)
        
        self.window_size = 32
        self.step = 10
        
    def forward(self, frames):
#         print(frames.shape)
        h_1 = None
#         print(h_1.shape)
        h_2 = None
        h_3 = None
        for t in range(int((len(frames[0][0][0])-self.window_size)/self.step)+1):
            frame = torch.from_numpy(frames[:, :, :, t*self.step: t*self.step+self.window_size])
#             print(frame.shape, h_1.shape)
            y, h_1 = self.GRCU_1(frame, h_1)
#             print(y.shape, h_1.shape)
            y, h_2 = self.GRCU_2(y, h_2)
            y, h_3 = self.GRCU_3(y, h_3)
#             print(y)
#         y = self.fc(y)
        y = self.fc(y.flatten())
#         print(y.shape)
        y = self.maxout(y.reshape(1, 480*2, 1, 1)).flatten()
#         print(y.shape)
        classes = self.fc_class(y)
        return self.softmax(classes)

In [410]:
model = GRCNN(10)
# for flac in db.index[:6]:
# #     print(flac)
#     with open("MGDs/"+flac["name"]+".npy", "rb") as file:
#         MGD = np.load(file).reshape(257, -1, 1)
#     with open("STFTs/"+flac["name"]+".npy", "rb") as file:
#         STFT = np.load(file).reshape(1, 1, 256, -1)
# #     print(MGD.shape, STFT[:, :, :, :31].shape)
# #     print(torch.from_numpy(STFT[:, :, :, :32]).shape)
# #     conv = nn.Conv2d(1, 16, (5, 5), stride=(1, 1), padding="same")
# #     output = conv(torch.from_numpy(STFT[:, :, :, :32]))
# #     print(output.shape)
#     outputs = model(STFT)
#     print(outputs.shape)
# print(os.chdir(".."))
with open("./STFTs/LA_T_1001169.npy", "rb") as file:
    STFT = np.load(file).reshape([1, 1, 256, -1])
STFT = (STFT - np.mean(STFT)) / np.std(STFT)
print(STFT)
output = model(STFT)
print(input.shape)
print(output.shape)
print(output)
# tensor([0.1120, 0.0947, 0.1091, 0.0973, 0.1256, 0.1022, 0.0839, 0.0925, 0.0881, 0.0944], grad_fn=<SoftmaxBackward>)

<ipython-input-376-ffea9b6469c0>:113: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.reset_gate.bias, 0.)
<ipython-input-376-ffea9b6469c0>:114: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.update_gate.bias, 0.)
<ipython-input-376-ffea9b6469c0>:115: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.out_gate.bias, 0.)


[[[[-0.6832037  -0.57169783 -0.55686945 ...  0.05938715 -0.13023205
    -0.7771059 ]
   [-0.9924119  -0.78656864 -0.61241287 ...  0.4642063  -0.32322216
    -0.77545226]
   [-0.32168755 -0.25977463 -0.19285311 ...  0.43186578  0.29145613
    -0.22066562]
   ...
   [ 1.8729985   1.9845482   1.8347391  ...  1.7656589   1.8738041
     1.8321075 ]
   [ 1.8718315   1.7921361   1.8735186  ...  1.87458     1.8177145
     1.8618876 ]
   [ 1.8169066   1.7729185   1.786942   ...  1.7825811   1.8691953
     1.8863722 ]]]]
torch.Size([20, 16, 50, 32])
torch.Size([10])
tensor([0.1072, 0.0777, 0.1008, 0.1017, 0.0991, 0.1220, 0.1018, 0.0909, 0.1032,
        0.0957], grad_fn=<SoftmaxBackward>)


In [67]:
frames = torch.randn(1, 30)
print(frames)
def con_frames(frames):
    for t in range(int((len(frames[0])-6)/2)+1):
        print(t)
        frame = frames[0][t*2: t*2+6]
        print(frame)
con_frames(frames)

tensor([[ 1.7909, -0.6654,  1.3372, -1.7296,  0.1955,  0.5792, -0.7481, -0.0869,
          0.0819,  1.9487,  2.2764, -0.7831,  1.0426,  0.5207,  0.1649,  0.8151,
         -0.0274,  1.5615, -0.8928,  0.3207, -1.3114,  0.4394, -1.1950,  0.2414,
          1.2117,  0.1229,  1.0073, -0.3994, -0.9846,  0.5623]])
0
tensor([ 1.7909, -0.6654,  1.3372, -1.7296,  0.1955,  0.5792])
1
tensor([ 1.3372, -1.7296,  0.1955,  0.5792, -0.7481, -0.0869])
2
tensor([ 0.1955,  0.5792, -0.7481, -0.0869,  0.0819,  1.9487])
3
tensor([-0.7481, -0.0869,  0.0819,  1.9487,  2.2764, -0.7831])
4
tensor([ 0.0819,  1.9487,  2.2764, -0.7831,  1.0426,  0.5207])
5
tensor([ 2.2764, -0.7831,  1.0426,  0.5207,  0.1649,  0.8151])
6
tensor([ 1.0426,  0.5207,  0.1649,  0.8151, -0.0274,  1.5615])
7
tensor([ 0.1649,  0.8151, -0.0274,  1.5615, -0.8928,  0.3207])
8
tensor([-0.0274,  1.5615, -0.8928,  0.3207, -1.3114,  0.4394])
9
tensor([-0.8928,  0.3207, -1.3114,  0.4394, -1.1950,  0.2414])
10
tensor([-1.3114,  0.4394, -1.1950,  0.2

In [58]:
x = np.arange(10)
print(x)
# with open('test.npy', 'wb') as f:
#     np.save(f, x)
    
with open("test.npy", "rb") as file:
    data = np.load(file)
print(data == x)

class Spectral_Features:
    def __init__(self, path):
        pass
        
    def extract_features(self, dest):
        pass

[0 1 2 3 4 5 6 7 8 9]
[ True  True  True  True  True  True  True  True  True  True]


In [8]:
# grp_phase = np.loadtxt("grp_phase.mat")
# cep = np.loadtxt("cep.mat")
# ts = np.loadtxt("ts.mat")
# print(grp_phase.shape)
# print(cep.shape)
# print(ts.shape)



import pandas as pd
import subprocess
import os
import torchaudio



path = "/media/ssd1T/antispoof/2019/LA"
# train_list = pd.read_csv(path, header=None, delim_whitespace=True)
# # print(train_list.at[0,])
# for row in train_list.iterrows():
#     print(row[1][1])
#     break
    
    
class Database:
    def __init__(self, index_path, data_path):
        self.index_path = index_path
        self.data_path = data_path
        
        self.index = self.generate_index(pd.read_csv(self.index_path, header=None, delim_whitespace=True))
        
    def generate_index(self, index_):
        index = []
        for row in index_.iterrows():
            index.append({"index": row[0], "id": row[1][0], "name": row[1][1], "label": {"type": row[1][3], "gender": row[1][4]}})
        return index
        
    def generate_MGD(self, dest, matlab_dir="matlab"):
        os.chdir(matlab_dir)    
        for flac in self.index:
#             print(os.getcwd())
            bashCommand = """octave --eval 'run("{}.flac", "{}", "{}")'""".format(self.data_path+"/"+flac["name"], flac["name"], "../"+dest)
#             print(bashCommand)
            process = os.system(bashCommand)
#             output, error = process.communicate()
#             print(process)
            grp_phase = np.loadtxt("../"+dest+"/"+flac["name"]+".mat")
#             print(grp_phase)
            with open("../"+dest+"/"+flac["name"]+'.npy', 'wb') as f:
                np.save(f, grp_phase)
            os.remove("../"+dest+"/"+flac["name"]+'.mat')
#             break
        os.chdir("..")
        
    def generate_STFT(self, dest):
        # print(waveform.numpy())
#         print(len(waveform[0]))
        n_fft = 1024
       
        window = "blackman"

        n_mels = 256
        fmin = 20
        fmax = 8000
        
        for flac in self.index:
            waveform, sample_rate = torchaudio.load(self.data_path+"/"+flac["name"]+".flac")
#             print(waveform)
            win_length = int(np.ceil(0.025*sample_rate))
            hop_length = int(np.ceil(0.010*sample_rate))
            X = librosa.stft(waveform.numpy()[0], n_fft, hop_length, win_length, window, center=True)
        #     print(np.abs(X).shape)
            frames = np.log(librosa.feature.melspectrogram(y=waveform.numpy()[0], sr=sample_rate, S=X, n_mels=n_mels, fmin=fmin, fmax=fmax) + 1e-6)
            with open("../"+dest+"/"+flac["name"]+'.npy', 'wb') as f:
                np.save(f, frames)

db = Database(path+"/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt", path+"/ASVspoof2019_LA_train/flac")
# db.generate_MGD("MGDs", "/home/tomtony/Sync/GRCNN/matlab")
# db.generate_STFT("STFTs")

In [40]:
a = torch.Tensor(5)
print(a)

tensor([2.7708e+20, 2.2945e+02, 3.9920e+24, 4.1996e+12, 8.9683e-44])


In [276]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.autograd import Function


class MaxoutConv(torch.nn.Module):
    """Maxout layer with convolution"""

    def __init__(self, in_channels,
                 out_channels, kernel_size, padding):
        """
        Define layers of maxout unit
        :param in_channels: number of channel of input convolution
        :type in_channels: :py:obj:`int`
        :param out_channels: number of channel of output convolution
        :type out_channels: :py:obj:`int`
        :param kernel_size: size of the weight matrix to convolve
        :type kernel_size: (:py:obj:`int`, :py:obj:`int`)
        """
        super(MaxoutConv, self).__init__()

        self.conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                    padding=padding)

        self.BN = torch.nn.BatchNorm2d(out_channels)

    def forward(self, _input, is_norm=False):
        """
        Pass the input to the maxout layer
        :param _input: input to the maxout layer
                       input is expected to have channel dimension
        :type _input: :py:class:`torch.Tensor`
        """
        z = self.conv(_input)
        if is_norm:
            z = self.BN(z)
        print("z.shape: ", z.shape)
        # (batch size, channels, height, width)
        h = torch.max(z, 1).values     # take max operation from first dimension(channel)
        # Insert 1 as channel dimension to h
        print("h.shape: ", h.shape)
        hshape = h.shape
        h = h.reshape(*([hshape[0]] + [1] + list(hshape[1:])))
        return h

input = torch.randn(16, 256, 32).reshape(1, 16, 256, 32)
# maxout = MaxoutConv(18, 9, 5, "same")
print(input.shape)
# output = maxout(input)
# print(output.shape)


# m = nn.Dropout(p=0.5)
# input = torch.randn(20, 16)
# output = m(input)
# print(input.shape)
# print(output.shape)




class Maxout(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input):
        x = input
        max_out=2    #Maxout Parameter
        kernels = x.shape[1]  # to get how many kernels/output
        feature_maps = int(kernels / max_out)
        out_shape = (x.shape[0], feature_maps, max_out, x.shape[2], x.shape[3])
#         print(x.shape)
        x= x.view(out_shape)
        print(x.shape)
        y, indices = torch.max(x[:, :, :], 2)
        ctx.save_for_backward(input)
        ctx.indices=indices
        ctx.max_out=max_out
        return y

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        input1,indices,max_out= ctx.saved_variables[0],Variable(ctx.indices),ctx.max_out
        input=input1.clone()
        for i in range(max_out):
            a0=indices==i
            input[:,i:input.data.shape[1]:max_out]=a0.float()*grad_output
      

        return input

maxout = Maxout.apply
output = maxout(input)
print(output.shape)
# print(input)
# print(output)

torch.Size([1, 16, 256, 32])
torch.Size([1, 8, 2, 256, 32])
torch.Size([1, 8, 256, 32])


In [384]:
a = torch.randn(1, 16, 256, 32)
b = torch.randn(1, 16, 256, 32)
torch.mul(a, b).shape

# pool of square window of size=3, stride=2
m = nn.MaxPool2d(3, stride=2)
# pool of non-square window
m = nn.MaxPool2d((2, 1), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m(input)
print(input.shape, output.shape)

x = torch.randn(1, 8, 32, 32)
fc = nn.Linear(8*32*32, 480*2)
y = fc(x)
print(y.shape)

torch.Size([20, 16, 50, 32]) torch.Size([20, 16, 25, 32])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x32 and 8192x960)

In [434]:
label = np.zeros(12)
# print(label)

import numpy as np
import torch
import torch.utils.data as data_utils

# Create toy data
x = np.linspace(start=1, stop=10, num=10)
x = np.array([np.random.normal(size=len(x)) for i in range(100)])
x = np.array([[1, 2, 3], [1], [2, 2, 2, 2, 2], [1, 1, 1], [3, 3, 3], [1, 2, 3], [1], [4, 2, 1, 1, 1, 1, 1], [1, 1, 1]])
# x = torch.from_numpy(x)
print(x.shape)
# Create DataLoader
dataset = torch.from_numpy(x).float()
print(x.shape)
dataloader = data_utils.DataLoader(dataset, batch_size=32)
batch = next(iter(dataloader))
print(batch.shape)

(9,)


<ipython-input-434-23c7a651f6f0>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array([[1, 2, 3], [1], [2, 2, 2, 2, 2], [1, 1, 1], [3, 3, 3], [1, 2, 3], [1], [4, 2, 1, 1, 1, 1, 1], [1, 1, 1]])


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [482]:
import datetime
now = datetime.datetime.now().strftime("%Y_%m_%d-%H%M")
print(now)

a = torch.Tensor()
b = torch.randn(3)
print(b)
a = torch.cat((a, b))
b = torch.randn(3)
print(b)
a = torch.cat((a, b))
print(a)

a = np.array([1, 2])
b = np.array([3, 4])
c = np.concatenate((a, b), axis=0)
print(c)


a = torch.randn(3)
b = torch.randn(3)
c = torch.cat([a, b])
print(c.shape)

m = nn.Dropout(p=0.2)
input = torch.randn(20, 1, 3)
output = m(input)
# print(output)

a = torch.randn(480)
b = torch.randn(480)
c = torch.stack((a, b))
print(c.shape)

2021_08_08-2125
tensor([-0.2889, -0.6835,  0.0709])
tensor([ 0.6768,  0.3287, -0.0517])
tensor([-0.2889, -0.6835,  0.0709,  0.6768,  0.3287, -0.0517])
[1 2 3 4]
torch.Size([6])
torch.Size([2, 480])


In [484]:
a = [1, "sad", 3]
for i, data in enumerate(a):
    print(i, data)

0 1
1 sad
2 3
